**You Can either run `liveInference.ipynb` file for this in the given github repository below, or first download all the requirements from that repo, then run this file.**

It supports -> `python==3.9`

In [ ]:
!git clone https://github.com/jaidh01/FastSAM.git

In [1]:
from fastsam import FastSAM, FastSAMPrompt
import torch 
import numpy as np
import cv2
import time

Expected end or semicolon (after name and no valid version specifier)
    git+https://github.com/openai/CLIP.git
       ^


In [2]:
model = FastSAM('FastSAM-x.pt')

c:\Users\DELL\FastSAM\ultralytics\nn\tasks.py:518: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [3]:
DEVICE = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(DEVICE)


cpu


In [6]:
cap = cv2.VideoCapture(0)

while cap.isOpened():

    suc, frame = cap.read()

    start = time.perf_counter()

    everything_results = model(
        source=frame,
        device=DEVICE,
        retina_masks=True,
        imgsz=1024,
        conf=0.4,
        iou=0.9)
    
    print(everything_results[0].masks.shape)
    print(everything_results[0].boxes.shape)
    print(everything_results[0].boxes[0].xyxy.cpu().numpy())

    # for box in everything_results[0].boxes:
    #     box = box.xyxy.cpu().numpy()[0]
    #     print(box)
    #     cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)

    # # everything_results = list(everything_results)
    # # print(everything_results[0].masks.shape)

    prompt_process = FastSAMPrompt(frame, everything_results, device=DEVICE)

    # # everything prompt
    # ann = prompt_process.everything_prompt()

    # text prompt
    #ann = prompt_process.text_prompt(text='a man')

    # bbox prompt
    # bbox default shape [0,0,0,0] -> [x1, y1, x2, y2]
    # bboxes default shape [[0,0,0,0]] -> [[x1, y1, x2, y2]]

    #ann = prompt_process.box_prompt(bbox=[200,200,300,300])
    #ann = prompt_process.box_prompt(bboxes=[[200,200,300,300], [500,500,600,600]])

    # Point prompt
    # points default [[0,0]] [[x1,y1], [x2,y2]]
    # point_label default [0] [1,0] 0:background, 1:foreground
    ann = prompt_process.point_prompt(points=[[620,360]], pointlabel=[1])

    print(f"ann: {ann}")

    end = time.perf_counter()
    total_time = end - start
    fps = 1 / total_time

    img = prompt_process.plot_to_result(img=frame, annotations=ann)

    #cv2.putText(img, f'FPS: ')

    cv2.imshow('frame', frame)
    cv2.imshow('img', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()



0: 768x1024 6 objects, 2418.4ms
Speed: 17.0ms preprocess, 2418.4ms inference, 8.2ms postprocess per image at shape (1, 3, 1024, 1024)


torch.Size([6, 480, 640])
torch.Size([6, 6])
[[     135.22      354.88       571.4         480]]
ann: [[[ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  ...
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]]]



0: 768x1024 6 objects, 2237.2ms
Speed: 17.2ms preprocess, 2237.2ms inference, 7.0ms postprocess per image at shape (1, 3, 1024, 1024)



torch.Size([6, 480, 640])
torch.Size([6, 6])
[[     136.25      356.07      571.16         480]]
ann: [[[ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  ...
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]]]


0: 768x1024 6 objects, 2205.3ms
Speed: 8.6ms preprocess, 2205.3ms inference, 7.5ms postprocess per image at shape (1, 3, 1024, 1024)



...

0: 768x1024 5 objects, 2238.2ms
Speed: 7.4ms preprocess, 2238.2ms inference, 6.6ms postprocess per image at shape (1, 3, 1024, 1024)



torch.Size([5, 480, 640])
torch.Size([5, 6])
[[     136.87      356.82      570.68         480]]
ann: [[[ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  ...
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]]]


0: 768x1024 5 objects, 2291.4ms
Speed: 8.2ms preprocess, 2291.4ms inference, 7.2ms postprocess per image at shape (1, 3, 1024, 1024)



torch.Size([5, 480, 640])
torch.Size([5, 6])
[[     136.95      354.49      571.14         480]]
ann: [[[ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  ...
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]
  [ True  True  True ...  True  True  True]]]


KeyboardInterrupt: 